<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_Pooled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Creates pooled data from several datasets to be used for prediction**

This notebook assembles CU histograms from multiple datasets for use in pooled prediction, a method in which lowly expressed genes can get a more stable CU histogram by "borrowing" information from similar datasets. We create two variants here - pooled data from full datasets and pooled data from downsampled dataset (downsampled 10 times). The reason for the latter is for use with down-sampling experiments, such as figure 3. We generate pooled data from 6 PBMC 10X datasets. For the downsampled variant, we downsample each dataset 10 times and pool a total of 60 datasets, since so much information is lost in the downsampling.

We also perform downsampling of a dataset to be able to compare prediction to sampling noise (fig S20)

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process and save the data


**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 982 (delta 62), reused 40 (delta 20), pack-reused 883
Receiving objects: 100% (982/982), 7.38 MiB | 19.84 MiB/s, done.
Resolving deltas: 100% (616/616), done.


In [2]:
#download BUG data from Zenodo
![ -d "figureData" ] && rm -r figureData
!mkdir figureData
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3_3.zip?download=1 && unzip 'PBMC_V3_3.zip?download=1' && rm 'PBMC_V3_3.zip?download=1'







--2020-07-02 18:38:41--  https://zenodo.org/record/3909758/files/PBMC_V3_3.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819306744 (1.7G) [application/octet-stream]
Saving to: ‘PBMC_V3_3.zip?download=1’

PBMC_V3_3.zip?downl 100%[===================>]   1.69G  11.0MB/s    in 2m 13s  

2020-07-02 18:40:55 (13.0 MB/s) - ‘PBMC_V3_3.zip?download=1’ saved [1819306744/1819306744]

Archive:  PBMC_V3_3.zip?download=1
   creating: PBMC_V3_3/
  inflating: PBMC_V3_3/Bug_10.RData  
  inflating: PBMC_V3_3/Bug_100.RData  
  inflating: PBMC_V3_3/Bug_20.RData  
  inflating: PBMC_V3_3/Bug_40.RData  
  inflating: PBMC_V3_3/Bug_5.RData   
  inflating: PBMC_V3_3/Bug_60.RData  
  inflating: PBMC_V3_3/Bug_80.RData  
  inflating: PBMC_V3_3/ds_summary.txt  
  inflating: PBMC_V3_3/pooledHist.RData  
  inflating: PBMC_V3_3/pooledHistDS.RData  
  inflating: PBMC_V3

In [3]:
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3_2.zip?download=1 && unzip 'PBMC_V3_2.zip?download=1' && rm 'PBMC_V3_2.zip?download=1'

--2020-07-02 18:41:16--  https://zenodo.org/record/3909758/files/PBMC_V3_2.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1558568885 (1.5G) [application/octet-stream]
Saving to: ‘PBMC_V3_2.zip?download=1’

PBMC_V3_2.zip?downl 100%[===================>]   1.45G  12.0MB/s    in 2m 21s  

2020-07-02 18:43:38 (10.6 MB/s) - ‘PBMC_V3_2.zip?download=1’ saved [1558568885/1558568885]

Archive:  PBMC_V3_2.zip?download=1
   creating: PBMC_V3_2/
  inflating: PBMC_V3_2/Bug_10.RData  
  inflating: PBMC_V3_2/Bug_100.RData  
  inflating: PBMC_V3_2/Bug_20.RData  
  inflating: PBMC_V3_2/Bug_40.RData  
  inflating: PBMC_V3_2/Bug_5.RData   
  inflating: PBMC_V3_2/Bug_60.RData  
  inflating: PBMC_V3_2/Bug_80.RData  
  inflating: PBMC_V3_2/ds_summary.txt  
  inflating: PBMC_V3_2/pooledHist.RData  
  inflating: PBMC_V3_2/pooledHistDS.RData  
  inflating: PBMC_V3

In [4]:
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3.zip?download=1 && unzip 'PBMC_V3.zip?download=1' && rm 'PBMC_V3.zip?download=1'

--2020-07-02 18:43:55--  https://zenodo.org/record/3909758/files/PBMC_V3.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1127576758 (1.0G) [application/octet-stream]
Saving to: ‘PBMC_V3.zip?download=1’

PBMC_V3.zip?downloa 100%[===================>]   1.05G  10.0MB/s    in 81s     

2020-07-02 18:45:17 (13.3 MB/s) - ‘PBMC_V3.zip?download=1’ saved [1127576758/1127576758]

Archive:  PBMC_V3.zip?download=1
   creating: PBMC_V3/
  inflating: PBMC_V3/Bug_10.RData    
  inflating: PBMC_V3/Bug_100.RData   
  inflating: PBMC_V3/Bug_20.RData    
  inflating: PBMC_V3/Bug_40.RData    
  inflating: PBMC_V3/Bug_5.RData     
  inflating: PBMC_V3/Bug_60.RData    
  inflating: PBMC_V3/Bug_80.RData    
  inflating: PBMC_V3/ds_summary.txt  
  inflating: PBMC_V3/pooledHist.RData  
  inflating: PBMC_V3/pooledHistDS.RData  
  inflating: PBMC_V3/PredEvalData.RDS

In [5]:
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_NG.zip?download=1 && unzip 'PBMC_NG.zip?download=1' && rm 'PBMC_NG.zip?download=1'

--2020-07-02 18:45:29--  https://zenodo.org/record/3909758/files/PBMC_NG.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1183970256 (1.1G) [application/octet-stream]
Saving to: ‘PBMC_NG.zip?download=1’

PBMC_NG.zip?downloa 100%[===================>]   1.10G  14.8MB/s    in 84s     

2020-07-02 18:46:54 (13.5 MB/s) - ‘PBMC_NG.zip?download=1’ saved [1183970256/1183970256]

Archive:  PBMC_NG.zip?download=1
   creating: PBMC_NG/
  inflating: PBMC_NG/Bug_10.RData    
  inflating: PBMC_NG/Bug_100.RData   
  inflating: PBMC_NG/Bug_20.RData    
  inflating: PBMC_NG/Bug_40.RData    
  inflating: PBMC_NG/Bug_5.RData     
  inflating: PBMC_NG/Bug_60.RData    
  inflating: PBMC_NG/Bug_80.RData    
  inflating: PBMC_NG/ds_summary.txt  
  inflating: PBMC_NG/pooledHist.RData  
  inflating: PBMC_NG/pooledHistDS.RData  
  inflating: PBMC_NG/PredEvalData.RDS

In [6]:
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_NG_2.zip?download=1 && unzip 'PBMC_NG_2.zip?download=1' && rm 'PBMC_NG_2.zip?download=1'

--2020-07-02 18:47:07--  https://zenodo.org/record/3909758/files/PBMC_NG_2.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1753743936 (1.6G) [application/octet-stream]
Saving to: ‘PBMC_NG_2.zip?download=1’

PBMC_NG_2.zip?downl 100%[===================>]   1.63G  15.7MB/s    in 2m 7s   

2020-07-02 18:49:15 (13.1 MB/s) - ‘PBMC_NG_2.zip?download=1’ saved [1753743936/1753743936]

Archive:  PBMC_NG_2.zip?download=1
   creating: PBMC_NG_2/
  inflating: PBMC_NG_2/Bug_10.RData  
  inflating: PBMC_NG_2/Bug_100.RData  
  inflating: PBMC_NG_2/Bug_20.RData  
  inflating: PBMC_NG_2/Bug_40.RData  
  inflating: PBMC_NG_2/Bug_5.RData   
  inflating: PBMC_NG_2/Bug_60.RData  
  inflating: PBMC_NG_2/Bug_80.RData  
  inflating: PBMC_NG_2/ds_summary.txt  
  inflating: PBMC_NG_2/pooledHist.RData  
  inflating: PBMC_NG_2/pooledHistDS.RData  
  inflating: PBMC_NG

In [7]:
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V2.zip?download=1 && unzip 'PBMC_V2.zip?download=1' && rm 'PBMC_V2.zip?download=1'

--2020-07-02 18:49:35--  https://zenodo.org/record/3909758/files/PBMC_V2.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 816179326 (778M) [application/octet-stream]
Saving to: ‘PBMC_V2.zip?download=1’

PBMC_V2.zip?downloa 100%[===================>] 778.37M  13.3MB/s    in 53s     

2020-07-02 18:50:28 (14.7 MB/s) - ‘PBMC_V2.zip?download=1’ saved [816179326/816179326]

Archive:  PBMC_V2.zip?download=1
   creating: PBMC_V2/
  inflating: PBMC_V2/Bug_10.RData    
  inflating: PBMC_V2/Bug_100.RData   
  inflating: PBMC_V2/Bug_20.RData    
  inflating: PBMC_V2/Bug_40.RData    
  inflating: PBMC_V2/Bug_5.RData     
  inflating: PBMC_V2/Bug_60.RData    
  inflating: PBMC_V2/Bug_80.RData    
  inflating: PBMC_V2/ds_summary.txt  
  inflating: PBMC_V2/pooledHist.RData  
  inflating: PBMC_V2/pooledHistDS.RData  
  inflating: PBMC_V2/PredEvalData.RDS  


In [8]:
!cd figureData && wget https://zenodo.org/record/3909758/files/EVALPBMC.zip?download=1 && unzip 'EVALPBMC.zip?download=1' && rm 'EVALPBMC.zip?download=1'


--2020-07-02 18:50:38--  https://zenodo.org/record/3909758/files/EVALPBMC.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 467646609 (446M) [application/octet-stream]
Saving to: ‘EVALPBMC.zip?download=1’

EVALPBMC.zip?downlo 100%[===================>] 445.98M  11.8MB/s    in 31s     

2020-07-02 18:51:10 (14.2 MB/s) - ‘EVALPBMC.zip?download=1’ saved [467646609/467646609]

Archive:  EVALPBMC.zip?download=1
   creating: EVALPBMC/
  inflating: EVALPBMC/Bug_10.RData   
  inflating: EVALPBMC/Bug_100.RData  
  inflating: EVALPBMC/Bug_20.RData   
  inflating: EVALPBMC/Bug_40.RData   
  inflating: EVALPBMC/Bug_5.RData    
  inflating: EVALPBMC/Bug_60.RData   
  inflating: EVALPBMC/Bug_80.RData   
  inflating: EVALPBMC/ds_summary.txt  
  inflating: EVALPBMC/pooledHist.RData  
  inflating: EVALPBMC/pooledHistDS.RData  
  inflating: EVALPBMC/PredEvalDa

In [9]:
#Check that download worked
!cd figureData && ls -l && cd PBMC_V3_2 && ls -l

total 28
drwxr-xr-x 2 root root 4096 Jul  1 20:29 EVALPBMC
drwxr-xr-x 2 root root 4096 Jul  1 22:18 PBMC_NG
drwxr-xr-x 2 root root 4096 Jul  1 22:37 PBMC_NG_2
drwxr-xr-x 2 root root 4096 Jul  1 22:52 PBMC_V2
drwxr-xr-x 2 root root 4096 Jul  1 21:27 PBMC_V3
drwxr-xr-x 2 root root 4096 Jul  1 21:43 PBMC_V3_2
drwxr-xr-x 2 root root 4096 Jul  1 22:03 PBMC_V3_3
total 1628004
-rw-r--r-- 1 root root 352633388 Jun 30 23:44 Bug_100.RData
-rw-r--r-- 1 root root 117036831 Jun 30 23:19 Bug_10.RData
-rw-r--r-- 1 root root 187888424 Jun 30 23:22 Bug_20.RData
-rw-r--r-- 1 root root 267712541 Jun 30 23:27 Bug_40.RData
-rw-r--r-- 1 root root  65984760 Jun 30 23:17 Bug_5.RData
-rw-r--r-- 1 root root 311852745 Jun 30 23:33 Bug_60.RData
-rw-r--r-- 1 root root 335390002 Jun 30 23:40 Bug_80.RData
-rw-r--r-- 1 root root      1065 Jul  1 21:24 ds_summary.txt
-rw-r--r-- 1 root root    256079 Jul  1 14:34 pooledHistDS.RData
-rw-r--r-- 1 root root    518870 Jul  1 14:35 pooledHist.RData
-rw-r--r-- 1 root root  2

**2. Prepare the R environment**

In [10]:
#switch to R mode
%reload_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [11]:
#install the R packages
%%R
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process and save the data**

Here we generate the pooled CU histograms, both from full datasets and downsampled ones. In addition, we calculate the average expression when downsampling 20 times, which will to a large extent remove the sampling noise. We then estimate the sampling noise as the difference between this mean and data that is downsampled only once.

In [12]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [13]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "ButterflyHelpers.R"))

#generate pooled histograms (for fig 3C)

loadBug("PBMC_V3")
generatePooledHistogramDS("PBMC_V3")
generatePooledHistogram("PBMC_V3") #these are not really used, but we generate them anyway, it is little extra effort
rmBug("PBMC_V3")

loadBug("PBMC_V3_2")
generatePooledHistogramDS("PBMC_V3_2")
generatePooledHistogram("PBMC_V3_2")
rmBug("PBMC_V3_2")

loadBug("PBMC_V3_3")
generatePooledHistogramDS("PBMC_V3_3")
generatePooledHistogram("PBMC_V3_3")
#also downsample 20 times of PBMC_V3_3 - to enable comparison with sampling noise for fig S20
PBMC_V3_3_ds10_20Times = downSampleBUGNTimes(getBug("PBMC_V3_3", 1), 0.1, 20)
save(PBMC_V3_3_ds10_20Times, file=paste0(figure_data_path, "PBMC_V3_3_ds10_20Times.RData"))
rmBug("PBMC_V3_3")

loadBug("PBMC_NG")
generatePooledHistogramDS("PBMC_NG")
generatePooledHistogram("PBMC_NG")
rmBug("PBMC_NG")

loadBug("PBMC_NG_2")
generatePooledHistogramDS("PBMC_NG_2")
generatePooledHistogram("PBMC_NG_2")
rmBug("PBMC_NG_2")

loadBug("PBMC_V2")
generatePooledHistogramDS("PBMC_V2")
generatePooledHistogram("PBMC_V2")
rmBug("PBMC_V2")

loadBug("EVALPBMC")
generatePooledHistogramDS("EVALPBMC")
generatePooledHistogram("EVALPBMC")
rmBug("EVALPBMC")



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] "Preparing..."
[1] 1
[1] "1:1"
[1] "1:2"
[1] "1:3"
[1] "1:4"
[1] "1:5"
[1] 2
[1] "2:1"
[1] "2:2"
[1] "2:3"
[1] "2:4"
[1] "2:5"
[1] 3
[1] "3:1"
[1] "3:2"
[1] "3:3"
[1] "3:4"
[1] "3:5"
[1] 4
[1] "4:1"
[1] "4:2"
[1] "4:3"
[1] "4:4"
[1] "4:5"
[1] 5
[1] "5:1"
[1] "5:2"
[1] "5:3"
[1] "5:4"
[1] "5:5"
[1] 6
[1] "6:1"
[1] "6:2"
[1] "6:3"
[1] "6:4"
[1] "6:5"
[1] 7
[1] "7:1"
[1] "7:2"
[1] "7:3"
[1] "7:4"
[1] "7:5"
[1] 8
[1] "8:1"
[1] "8:2"
[1] "8:3"
[1] "8:4"
[1] "8:5"
[1] 9
[1] "9:1"
[1] "9:2"
[1] "9:3"
[1] "9:4"
[1] "9:5"
[1] 10
[1] "10:1"
[1] "10:2"
[1] "10:3"
[1] "10:4"
[1] "10:5"
[1] "Preparing..."
[1] 1
[1] "1:1"
[1] "1:2"
[1] "1:3"
[1] "1:4"
[1] "1:5"
[1] 2
[1] "2:1"
[1] "2:2"
[1] "2:3"
[1] "2:4"
[1] "2:5"
[1] 3
[1] "3:1"
[1] "3:2"
[1] "3:3"
[1] "3:4"
[1] "3:5"
[1] 4
[1] "4:1"
[1] "4:2"
[1] "4:3"
[1] "4:4"
[1] "4:5"
[1] 5
[1] "5:1"
[1] "5:2"
[1] "5:3"
[1] "5:4"
[1] "5:5"
[1] 6
[1] "6:1"
[1] "6:2"
[1] "6:3"
[1] "6:4"
[1] "6:5"
[1] 7
[1] "7:1"
[1] "7:2"
[1] "7:3"
[1] "7:4"
[1] "7:5"
[1] 

R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] 2


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



...

[1] 19


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] 20


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "Preparing..."
[1] 1
[1] "1:1"
[1] "1:2"
[1] "1:3"
[1] "1:4"
[1] "1:5"
[1] 2
[1] "2:1"
[1] "2:2"
[1] "2:3"
[1] "2:4"
[1] "2:5"
[1] 3
[1] "3:1"
[1] "3:2"
[1] "3:3"
[1] "3:4"
[1] "3:5"
[1] 4
[1] "4:1"
[1] "4:2"
[1] "4:3"
[1] "4:4"
[1] "4:5"
[1] 5
[1] "5:1"
[1] "5:2"
[1] "5:3"
[1] "5:4"
[1] "5:5"
[1] 6
[1] "6:1"
[1] "6:2"
[1] "6:3"
[1] "6:4"
[1] "6:5"
[1] 7
[1] "7:1"
[1] "7:2"
[1] "7:3"
[1] "7:4"
[1] "7:5"
[1] 8
[1] "8:1"
[1] "8:2"
[1] "8:3"
[1] "8:4"
[1] "8:5"
[1] 9
[1] "9:1"
[1] "9:2"
[1] "9:3"
[1] "9:4"
[1] "9:5"
[1] 10
[1] "10:1"
[1] "10:2"
[1] "10:3"
[1] "10:4"
[1] "10:5"
[1] "Preparing..."
[1] 1
[1] "1:1"
[1] "1:2"
[1] "1:3"
[1] "1:4"
[1] "1:5"
[1] 2
[1] "2:1"
[1] "2:2"
[1] "2:3"
[1] "2:4"
[1] "2:5"
[1] 3
[1] "3:1"
[1] "3:2"
[1] "3:3"
[1] "3:4"
[1] "3:5"
[1] 4
[1] "4:1"
[1] "4:2"
[1] "4:3"
[1] "4:4"
[1] "4:5"
[1] 5
[1] "5:1"
[1] "5:2"
[1] "5:3"
[1] "5:4"
[1] "5:5"
[1] 6
[1] "6:1"
[1] "6:2"
[1] "6:3"
[1] "6:4"
[1] "6:5"
[1] 7
[1] "7:1"
[1] "7:2"
[1] "7:3"
[1] "7:4"
[1] "7:5"
[1] 

In [14]:
!cd figureData && ls -l 

total 160
drwxr-xr-x 2 root root   4096 Jul  1 20:29 EVALPBMC
drwxr-xr-x 2 root root   4096 Jul  1 22:18 PBMC_NG
drwxr-xr-x 2 root root   4096 Jul  1 22:37 PBMC_NG_2
drwxr-xr-x 2 root root   4096 Jul  1 22:52 PBMC_V2
drwxr-xr-x 2 root root   4096 Jul  1 21:27 PBMC_V3
drwxr-xr-x 2 root root   4096 Jul  1 21:43 PBMC_V3_2
drwxr-xr-x 2 root root   4096 Jul  1 22:03 PBMC_V3_3
-rw-r--r-- 1 root root 133187 Jul  2 19:45 PBMC_V3_3_ds10_20Times.RData
